# YOLO - Inferencia
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jun 2021<br>
**Descripción:** Notebook para inferir resultados.

## Instalación de OpenCV

``` bash
    sudo apt install python3-opencv
    sudo apt-get install libopencv-dev
```

## Instalar zip

``` bash
    sudo apt install zip unzip
``` 

## Preparación del entorno

* Crear entorno virtual (solo la primera vez) dentro de la carpeta `/inference`.

``` bash
    python3 -m venv yolo_inference --system-site-packages # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source yolo_inference/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r env # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

### Probar importar las siguientes librerias, para saber si todo el procedimiento fue correcto.

In [ ]:
import numpy
import cv2

## Clonación y construcción de Darknet (por única vez)

In [ ]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

## Compilando Darknet

Modificar el archivo `/darknet/Makefile`. Una opción es abrirlo directamente o usando las siguientes instrucciones.

In [ ]:
# cambie el makefile para tener GPU y OPENCV habilitados
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=1/GPU=0/' Makefile
!sed -i 's/CUDNN=1/CUDNN=0/' Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile

Modificar las siguientes lineas para redireccionar el path de CUDA

```
GPU=1
CUDNN=1

ifeq ($(GPU), 1)
COMMON+= -DGPU -I/usr/local/cuda-11.4/include/
CFLAGS+= -DGPU
ifeq ($(OS),Darwin) #MAC
LDFLAGS+= -L/usr/local/cuda-11.4/lib -lcuda -lcudart -lcublas -lcurand
else
LDFLAGS+= -L/usr/local/cuda-11.4/lib64 -lcuda -lcudart -lcublas -lcurand
endif
endif

ifeq ($(CUDNN), 1)
COMMON+= -DCUDNN
ifeq ($(OS),Darwin) #MAC
CFLAGS+= -DCUDNN -I/usr/local/cuda-11.4/include
LDFLAGS+= -L/usr/local/cuda-11.4/lib -lcudnn
else
CFLAGS+= -DCUDNN -I/usr/local/cudnn/include
LDFLAGS+= -L/usr/local/cudnn/lib64 -lcudnn
endif
endif
```

> **Nota:** Tambien se puede usar el archivo `Makefile_mod` para reemplazar (recordar modificar el nombre)

In [2]:
%cd darknet

/home/maximiliano/ml_uali/inference/darknet


In [ ]:
# make darknet (crea darknet para que luego pueda usar el archivo ejecutable darknet para ejecutar o entrenar detectores de objetos)
!make -h

In [5]:
!make -d

GNU Make 4.2.1
Built for x86_64-pc-linux-gnu
Copyright (C) 1988-2016 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Reading makefiles...
Reading makefile 'Makefile'...
Updating makefiles....
 Considering target file 'Makefile'.
  Looking for an implicit rule for 'Makefile'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.o'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.c'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.cc'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.C'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.cpp'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.p'.
  Try

In [6]:
./darknet detector test ./cfg/coco.data ./cfg/yolov4.cfg ./yolov4.weights

SyntaxError: invalid syntax (1106219655.py, line 1)

## Descargando pesos de YOLOv4

### Entrenamiento propio

yolov4-obj_best.weights URL: https://drive.google.com/file/d/1-5eprW8D2Si3gZOqaN4QadHOFhvu6OWT/view?usp=sharing

yolov4-obj_final.weights URL: https://drive.google.com/file/d/1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H/view?usp=sharing

In [ ]:
# yolov4-obj_best.weights
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H" -O ../yolov4-obj_final.weights && rm -rf /tmp/cookies.txt

### Descargando data_training

In [ ]:
# descargar comprimido de data_training
!wget -O ../data_training.zip https://github.com/githubuali/ml_uali/blob/main/training/yolo_v1/data_training.zip?raw=true

# descomprima los conjuntos de datos 
!unzip ../data_training.zip -d ../data_training

# copiar obj.name
!cp ../data_training/obj.names data

# eliminando carpetas
!rm -rf ../data_training.zip

### Ejecutando Darknet y YOLOv4 (con imagenes por URL)

URL: https://drive.google.com/file/d/1aLTw006n7dTYk_9P8TsY7tN-3xd-txLW/view?usp=sharing

In [ ]:
# descargar comprimido de imagenes de prueba
!wget -O ../image_ypf.zip https://github.com/githubuali/ml_uali/blob/main/data/image_ypf.zip?raw=true

# descomprima los conjuntos de datos 
!unzip ../image_ypf.zip

# eliminando carpetas
!rm -rf ../image_ypf.zip

In [ ]:
# Necesitamos configurar nuestro cfg personalizado en modo de prueba (unica vez)
!sed -i 's/batch=64/batch=1/' ../data_training/yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' ../data_training/yolov4-obj.cfg

In [8]:
import time
inicio = time.time()

time.strftime('La inferencia se inició: '+'%Y-%m-%d %H:%M:%S', time.localtime())

'La inferencia se inició: 2021-07-28 16:05:02'

In [13]:
#ejecute su detector personalizado con este comando (cargue una imagen en su unidad de Google para probar, la bandera thresh establece la precisión que debe tener la detección para mostrarla)
!cd darknet && ./darknet detector test ../data_training/obj.data ../data_training/yolov4-obj.cfg ../yolov4-obj_final.weights -ext_output -dont_show -out result.json < ../eventos/eventosDetectados_YPF/images.txt 

time.strftime('La inferencia finalizó: '+'%Y-%m-%d %H:%M:%S', time.localtime())

fin = time.time()
print('El tiempo total de proceso es de: ' + str(fin-inicio/60) + ' minutos.') # 1.5099220275878906

CUDA status Error: file: ./src/dark_cuda.c : () : line: 39 : build time: Jul 13 2021 - 16:03:05 

 CUDA Error: no CUDA-capable device is detected
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
darknet: ./src/utils.c:331: error: Assertion `0' failed.
/bin/bash: line 1:  8076 Aborted                 (core dumped) ./darknet detector test ../data_training/obj.data ../data_training/yolov4-obj.cfg ../yolov4-obj_final.weights -ext_output -dont_show -out result.json < ../eventos/eventosDetectados_YPF/images.txt
El tiempo total de proceso es de: 1600363567.1995637 minutos.
